In [1]:
import pandas as pd
import hopsworks
import joblib
import openmeteo_requests
import requests_cache
from retry_requests import retry
from datetime import datetime
import os

def get_weather_forecast():
    print("Connecting to Open-Meteo Forecast API for future weather...")
    
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Fetch 7-day forecast for Flores Island
    # Note: Parameters must match training features (excluding target 'wind_speed')
    params = {
        "latitude": 39.4532,
        "longitude": -31.1274,
        "daily": ["temperature_2m_max", "precipitation_sum", "wind_gusts_10m_max", "wind_direction_10m_dominant"],
        "timezone": "Atlantic/Azores",
        "forecast_days": 7
    }
    
    url = "https://api.open-meteo.com/v1/forecast"
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]

    daily = response.Daily()
    daily_data = {
        "date": pd.date_range(
            start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
            end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = daily.Interval()),
            inclusive = "left"
        )
    }
    
    # Map to feature names used during training
    daily_data["temperature_max"] = daily.Variables(0).ValuesAsNumpy()
    daily_data["precipitation"] = daily.Variables(1).ValuesAsNumpy()
    daily_data["wind_gusts"] = daily.Variables(2).ValuesAsNumpy()
    daily_data["wind_direction"] = daily.Variables(3).ValuesAsNumpy()
    
    df = pd.DataFrame(data = daily_data)
    df['date_str'] = df['date'].dt.strftime('%Y-%m-%d')
    
    return df

def run_inference():
    # 1. Login to Hopsworks
    print("Logging into Hopsworks...")
    project = hopsworks.login()
    
    # 2. Download Model
    print("Downloading model from Registry...")
    mr = project.get_model_registry()
    model = mr.get_model(name="azores_wind_model", version=1)
    model_dir = model.download()
    
    model_path = os.path.join(model_dir, "azores_wind_model.pkl")
    trained_model = joblib.load(model_path)
    
    # 3. Get Weather Forecast
    df_forecast = get_weather_forecast()
    print(f"Fetched {len(df_forecast)} days of forecast data.")
    
    # 4. Prepare Features
    features = df_forecast[['temperature_max', 'precipitation', 'wind_gusts', 'wind_direction']]
    
    # 5. Predict Wind Speed
    print("Predicting wind speed...")
    predictions = trained_model.predict(features)
    
    # 6. Display Results: Check for Boat Cancellation
    # Assumption: Speedboats cancel if wind > 30 km/h
    WIND_LIMIT = 30.0 
    
    print("\n Flores -> Corvo Boat Forecast:")
    print("-" * 60)
    for date, wind, gust in zip(df_forecast['date_str'], predictions, df_forecast['wind_gusts']):
        wind = max(0, wind) # Fix potential negative numbers
        
        if wind < WIND_LIMIT:
            status = "Boat Running (Go)"
        else:
            status = "High Wind / Cancelled"
            
        print(f"{date} | Pred Wind: {wind:4.1f} km/h (Gusts {gust:.1f}) | {status}")
    print("-" * 60)

if __name__ == "__main__":
    run_inference()

🔐 Logging into Hopsworks...
2026-01-13 22:28:59,583 INFO: Initializing external client
2026-01-13 22:28:59,583 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-13 22:29:00,235 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2026-01-13 22:29:01,176 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1303706
📥 Downloading model from Registry...


Downloading: 0.000%|          | 0/203984 elapsed<00:00 remaining<?

📡 Connecting to Open-Meteo Forecast API for future weather...
🌦️ Fetched 7 days of forecast data.
🔮 Predicting wind speed...
2026-01-13 22:29:04,897 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-13 22:29:04,897 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-13 22:29:04,898 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.


🚤 Flores -> Corvo Boat Forecast:
------------------------------------------------------------
📅 2026-01-13 | Pred Wind: 29.2 km/h (Gusts 49.7) | ✅ Boat Running (Go)
📅 2026-01-14 | Pred Wind: 48.8 km/h (Gusts 66.2) | ❌ High Wind / Cancelled
📅 2026-01-15 | Pred Wind: 27.1 km/h (Gusts 42.1) | ✅ Boat Running (Go)
📅 2026-01-16 | Pred Wind: 14.7 km/h (Gusts 28.8) | ✅ 